In [19]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Dataset

In [24]:
iris=datasets.load_iris()
X=iris['data']
y=iris['target']
X_train_iris,X_test_iris,y_train_iris,y_test_iris=train_test_split(X,y,test_size=0.2)
standardscalar=StandardScaler()
standardscalar.fit(X_train_iris)
X_train_iris_standard=standardscalar.transform(X_train_iris)
X_test_iris_standard=standardscalar.transform(X_test_iris)
#X_train_iris[:10],X_train_iris_standard[:10]

In [23]:
digits=datasets.load_digits()
X=digits.data
y=digits.target
X_train_digits,X_test_digits,y_train_digits,y_test_digits=train_test_split(X,y)

## KNN

In [22]:
def KNN_classify(k,X_train,y_train,x):
    distances=[sqrt(np.sum((x_train-x)**2)) for x_train in X_train]
    neareat=np.argsort(distances)
    topk_y=y_train[neareat[:k]]
    votes=Counter(topk_y)
    return votes.most_common()[0][0]

## Eva

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
knn_clf=KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train_iris_standard,y_train_iris)
y_predict=knn_clf.predict(X_test_iris_standard)
acc=np.sum(y_predict==y_test_iris)/len(y_test_iris)
acc_sklean=knn_clf.score(X_test_iris_standard,y_test_iris)
acc,acc_sklean

(1.0, 1.0)

## grid search & cross validation

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
hyperparameters=[
    {'weights':['distance'],'n_neighbors':[i for i in range(2,11)],'p':[i for i in range(1,6)]}
]
knn_clf=KNeighborsClassifier()
grid_search=GridSearchCV(param_grid=hyperparameters,estimator=knn_clf,n_jobs=-1)
grid_search.fit(X_train_digits,y_train_digits)
best_knn_clf=grid_search.best_estimator_
grid_search.best_params_,best_knn_clf.score(X_test_digits,y_test_digits)

({'n_neighbors': 4, 'p': 3, 'weights': 'distance'}, 0.9844444444444445)

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
best_score,best_p,best_k=0,0,0
for k in range(2,11):
    for p in range(1,6):
        knn_clf=KNeighborsClassifier(weights='distance',p=p,n_neighbors=k)
        scores=cross_val_score(knn_clf,X_train_digits,y_train_digits)
        score=np.mean(scores)
        if score> best_score:
            best_score,best_p,best_k=score,p,k

In [33]:
best2_knn_clf=KNeighborsClassifier(weights='distance',p=best_p,n_neighbors=best_k)
best2_knn_clf.fit(X_train_digits,y_train_digits)
best_k,best_p,best2_knn_clf.score(X_test_digits,y_test_digits)

(4, 3, 0.9844444444444445)